In [15]:
from helpers.data import get_dataloaders
from helpers.train import TrainingManager
from helpers.loss_accuracy import accuracy
import models.resnet
import torch
from torch.optim import Adam
from torch.nn import CrossEntropyLoss

In [9]:
trainloader, testloader = get_dataloaders('cifar10', 32)

Files already downloaded and verified


In [10]:
device = torch.device('cuda')

In [11]:
model = models.resnet.resnet1202()
model = model.to(device)
optimizer = Adam(model.parameters(), 2e-4)

In [12]:
trial_name = "resnet1202_adam_2e-4"

In [13]:
tm = TrainingManager(trial_name, load=False, is_trial=True)

In [14]:
tm.train(model, optimizer, 
         trainloader, testloader, 
         CrossEntropyLoss(), CrossEntropyLoss(), 
         accuracy, accuracy, device=device, no_iterations=100000)

  0%|          | 0/100000 [00:00<?, ?it/s]

Start training trial: resnet20_adam_2e-4


{tr_loss: 1.79483, tr_acc: 0.33250, te_loss: 1.70046, te_acc: 0.38375}:   1%|          | 596/100000 [00:21<58:50, 28.15it/s]   


KeyboardInterrupt: 